In [ ]:
using PyBokeh

In [ ]:
ENV["SEEP_NO_GPU"]=false
using Seep, JLD
T = Float32
relu(x::ANode) = Seep.max!(0, x)
make_node(a::Array) = ANode(a)
make_node(x::Int...) = ANode(x...)
get_data(x) = x

include("data.jl")
data = IRData()
nothing

In [ ]:
@load "snapshot-rnn/statistics.jld"
f = line(figure(), collect(1:length(E)), E)
ylim(f, 0, 1)

In [ ]:
# Constants
const sequence_length = 50
const input_size = 2
const hidden_size = [10, 10]
const feature_size = 5
const output_size = 2
const batch_size = size(data.test, 3)

# Network
const X = ANode[make_node(input_size, batch_size) for i=1:sequence_length]
const Y = make_node(output_size, batch_size)
y = make_node(zeros(T, output_size, batch_size))

params = Flow[]
let 
    # Initialize nodes
    fe = Linear("e", input_size, hidden_size[1])
    fh = Flow[]
    for j = 1:length(hidden_size)
        k = j==1 ? 1 : j-1

        f = LSTM("h$(j)", hidden_size[k], hidden_size[j])
        push!(fh, f)
    end
    ff = Linear("features", hidden_size[end], feature_size)
    fy = Linear("output", feature_size, output_size)
    push!(params, fe, fh..., ff, fy)

    # Initialize LSTM variables
    h = ANode[make_node(zeros(T, hidden_size[i], batch_size)) for i = 1:length(hidden_size)]
    c = ANode[make_node(zeros(T, hidden_size[i], batch_size)) for i = 1:length(hidden_size)]

    # Build graph
    x = make_node(zeros(T, input_size, batch_size))
    for i = 1:length(X)
        x = relu(fe(X[i]))
        for j = 1:length(hidden_size)
            c[j], h[j] = fh[j](x, c[j], h[j])
            x = h[j]
        end
    end
    x = tanh(ff(x))
    y = softmax(fy(x), 1)
end

# Gradient Calculation for SGD
RNN = instance(y)
nothing

In [ ]:
# Load Latest Snapshot
n_epoch_start = 0

if isdir("snapshot-brnn")
    f = filter(x->contains(x, "snapshot-"), readdir("snapshot-rnn"))
    if length(f) > 0
        epochs = map(x->parse(Int, split(split(x, ".")[1], "-")[end]), f)
        ind = indmax(epochs)
        n_epoch_start = epochs[ind]
        @show n_epoch_start
        load_snapshot(joinpath("snapshot-rnn", f[ind]), y)
    end
end


In [ ]:
function eval_test()
    xx = data.test

    for i = 1:length(X)
        xi = RNN[X[i]]
        xi[:, :] = xx[:, i, :]
    end

    RNN()
    RNN[y]
end

In [ ]:
ys = eval_test()
;

In [ ]:
hist(figure(), vec(ys[1, :]), line_color="black")